In [9]:
import pandas as pd
import re
import spacy
import numpy as np
import gzip
import gensim.downloader
import torch
from torch.utils.data import Dataset, DataLoader
import pickle
from tqdm import tqdm

In [2]:
# train = pd.read_csv("./nlp-getting-started/train.csv")
# test = pd.read_csv("./nlp-getting-started/test.csv")

# train["isTrain"] = True
# test["isTrain"] = False

# full = pd.concat([train, test])
# # print(full)

# def clean_text(row):
#     clean = row["text"]
    
#     if len(row["tags"]) != 0:
#         for word in row["tags"]:
#             clean = clean.replace(word, "")
    
#     if len(row["links"]) != 0:
#         for word in row["links"]:
#             clean = clean.replace(word, "")
    
#     #only remove the # symbol
#     clean = clean.replace("#", "").replace("/", "").replace("(", "").replace(")", "")
    
#     return clean.strip()

# def get_at(row):
#     return re.findall("@[\w]+", row["text"])

# def get_http(row):
#     return re.findall("http[\:\/\.\w]+", row["text"])

# def get_hashtags(row):
#     return re.findall("#[\w]+", row["text"])

# def number_of_tags(row):
#     return len(row["tags"])

# def number_of_links(row):
#     return len(row["links"])

# def number_of_hashs(row):
#     return len(row["hashtags"])

# full["tags"] = full.apply(lambda row: get_at(row), axis = 1)
# full["links"] = full.apply(lambda row: get_http(row), axis = 1)
# full["hashtags"] = full.apply(lambda row: get_hashtags(row), axis = 1)

# full["number_of_tags"] = full.apply(lambda row: number_of_tags(row), axis = 1)
# full["number_of_links"] = full.apply(lambda row: number_of_links(row), axis = 1)
# full["number_of_hashs"] = full.apply(lambda row: number_of_hashs(row), axis = 1)

# full["clean_text"] = full.apply(lambda row: clean_text(row), axis = 1)
# full.sample(5)

In [3]:
with gzip.open('./test.txt.gz', 'rt') as f:
    lines = [x.rstrip('\n') for x in f.readlines()]
    
    sentences = [[]]
    idx = 0
    for line in lines:
        if line != '':
            sentences[idx].append(line.split()[0])
        else:
            sentences.append([])
            idx += 1
            
#     print(sentences[:100])
    
    for sentence in sentences:
        for i, word in enumerate(sentence):
            if word == "'s":
                sentence[i-1] = sentence[i-1] + "'s"
                del sentence[i]
                
    print(sentences[:100])
    print(len(sentences))

FileNotFoundError: [Errno 2] No such file or directory: './test.txt.gz'

In [6]:
jpdy_data = pd.read_json("JEOPARDY_QUESTIONS1.json")

"'For the last 8 years of his life, Galileo was under house arrest for espousing this man's theory'"

In [7]:
sentences = [sent.strip().split(' ') for sent in jpdy_data['question'] if sent.strip()]

In [10]:
nlp = spacy.load('en_core_web_sm')

sub_toks = []
sub_indices = []

new_sentences = []

for i, sentence in tqdm(enumerate(sentences)):
    sent = ' '.join(sentence)
#     print(sent)
    doc = nlp(sent)
    temp = [tok for tok in doc if (tok.dep_ == "nsubj")]
    if len(temp) == 1:
        if str(temp[0]) in sentence:
            new_sentences.append(sentence)
            sub_toks.append(temp[0])
            sub_indices.append(sentence.index(str(temp[0])))
            
    if i + 1 % 200 == 0:
        print('{} out of {} done'.format(i+1, len(sentences)))
    
print(sub_toks[:10])
print(sub_indices[:10])
print(new_sentences[:10])

216930it [24:46, 145.91it/s]

[Galileo, city, company, insect, inches, merchandise, we, I, name, temperature]
[8, 1, 9, 8, 8, 9, 2, 3, 9, 2]
[["'For", 'the', 'last', '8', 'years', 'of', 'his', 'life,', 'Galileo', 'was', 'under', 'house', 'arrest', 'for', 'espousing', 'this', "man's", "theory'"], ["'The", 'city', 'of', 'Yuma', 'in', 'this', 'state', 'has', 'a', 'record', 'average', 'of', '4,055', 'hours', 'of', 'sunshine', 'each', "year'"], ["'In", '1963,', 'live', 'on', '"The', 'Art', 'Linkletter', 'Show",', 'this', 'company', 'served', 'its', 'billionth', "burger'"], ["'In", 'the', 'title', 'of', 'an', 'Aesop', 'fable,', 'this', 'insect', 'shared', 'billing', 'with', 'a', "grasshopper'"], ["'In", 'the', 'winter', 'of', '1971-72,', 'a', 'record', '1,122', 'inches', 'of', 'snow', 'fell', 'at', 'Rainier', 'Paradise', 'Ranger', 'Station', 'in', 'this', "state'"], ["'This", 'housewares', 'store', 'was', 'named', 'for', 'the', 'packaging', 'its', 'merchandise', 'came', 'in', '&', 'was', 'first', 'displayed', "on'"], ['\

In [14]:
print(len(new_sentences))
print(len(sub_toks))
print(len(sub_indices))
split = len(new_sentences)//10*9
print(f'Split:{split}')

print(new_sentences[:10])
print(sub_toks[:10])
print(sub_indices[:10])

pickle.dump({'sentences': new_sentences[:split], 'sub_indices': sub_indices[:split]}, open('subdata_jpdy_train.pkl', 'wb'))
pickle.dump({'sentences': new_sentences[split:], 'sub_indices': sub_indices[split:]}, open('subdata_jpdy_test.pkl', 'wb'))

78503
78503
78503
Split:70650
[["'For", 'the', 'last', '8', 'years', 'of', 'his', 'life,', 'Galileo', 'was', 'under', 'house', 'arrest', 'for', 'espousing', 'this', "man's", "theory'"], ["'The", 'city', 'of', 'Yuma', 'in', 'this', 'state', 'has', 'a', 'record', 'average', 'of', '4,055', 'hours', 'of', 'sunshine', 'each', "year'"], ["'In", '1963,', 'live', 'on', '"The', 'Art', 'Linkletter', 'Show",', 'this', 'company', 'served', 'its', 'billionth', "burger'"], ["'In", 'the', 'title', 'of', 'an', 'Aesop', 'fable,', 'this', 'insect', 'shared', 'billing', 'with', 'a', "grasshopper'"], ["'In", 'the', 'winter', 'of', '1971-72,', 'a', 'record', '1,122', 'inches', 'of', 'snow', 'fell', 'at', 'Rainier', 'Paradise', 'Ranger', 'Station', 'in', 'this', "state'"], ["'This", 'housewares', 'store', 'was', 'named', 'for', 'the', 'packaging', 'its', 'merchandise', 'came', 'in', '&', 'was', 'first', 'displayed', "on'"], ['\'"And', 'away', 'we', 'go"\''], ["'In", '1000', 'Rajaraja', 'I', 'of', 'the', 'Ch

In [5]:
print(list(gensim.downloader.info()['models'].keys()))

glove = gensim.downloader.load('glove-wiki-gigaword-200')

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


KeyboardInterrupt: 

In [ ]:
a = glove[['augusts', 'bye']]
print(a.shape)
print('Hi' in glove)

In [64]:
class WordDataset(Dataset):
    
    def __init__(self, sfile):
        d = pickle.load(open(sfile, 'rb'))
        
        self.sentences = d['sentences']
        self.indices = d['sub_indices']
        
        for sent in self.sentences:
            for i, word in enumerate(sent):
                sent[i] = word.lower().replace("'", "")
                if sent[i] == '':
                    del sent[i]
        
        self.glove = gensim.downloader.load('glove-wiki-gigaword-200')
        
        self.sentence_embeddings = []
        
        for sent in self.sentences:
            temp = []
            for word in sent:
                if word in self.glove:
                    temp.append(self.glove[word])
            self.sentence_embeddings.append(np.array(temp))
            
    def __len__(self):
        return len(self.sentence_embeddings)
    
    def __getitem__(self, idx):
        return (self.sentences[idx], self.sentence_embeddings[idx], self.indices[idx])

In [65]:
train_dataset = WordDataset('subdata.pkl')
test_dataset = WordDataset('subdata_test.pkl')

In [68]:
print(train_dataset[1][1].shape)

(26, 200)
